El siguiente código genera un diccionario  ordenado para mapear posiciones a listas de genes afectadas por esta.

In [ ]:
import collections

#gen file path
genPossitions_File="D:\\Genomics\\ucsc_genome_bioinformatics_anotations_data\\ucscGenePfam.txt.gz"    
#gen position dictionary path
genPosDicFile="D:\\Genomics\\genomePositionsDictionary.pkl"


#funciones auxiliares

def chrtoint(chr):
    if chr[3]=='X': return 23 # 22 +x+y
    elif chr[3]=='Y': return 24
    elif chr[3]=='M': return 25   #mitocondrial ?
    elif chr[3]=='U': return 26   # ???
    else: return int(chr[3:])
def chrtointfrom0(chr):
    if chr[0]=='X': return 23 # 22 +x+y
    elif chr[0]=='Y': return 24
    elif chr[0]=='M': return 25   #mitocondrial ?
    elif chr[0]=='U': return 26   # ???
    else: return int(chr[0:])
#this value is used to generate a global position adding it to the chr number 
genmultiplier=1000000000000 





#transformations for filtering and loading of gen file
genPositionsRdd=(sc.textFile(genPossitions_File)
                .map(lambda x: x.split("\t"))
                .filter(lambda x: len(x[1].split("_"))==1)
                .map(lambda x: [chrtoint(x[1].split("_")[0])]+x[2:])
                .map(lambda x: x+[x[0]*1000000000000+int(x[1]),x[0]*1000000000000+int(x[2])]) 
                .zipWithIndex()
                .map(lambda x: (x[0][-2:],x[-1])) 
                )



#This section generates de gen position dictionary
genStartPossitionsList=[]
genEndPossitionsList=[]
genposList=genPositionsRdd.collect()
for item in genposList:
    genStartPossitionsList.append((item[0][0],item[1]))
    genEndPossitionsList.append((item[0][1],item[1]))
genStartPossitionsList.sort()
genEndPossitionsList.sort()
finalGenPosList=[]
tempDict={}
index1=0
index2=0
for pos in range(len(genStartPossitionsList)):
    index1=pos
    t2=index2
    changed=0
    for pos2 in range(index2,index1):
        if genEndPossitionsList[pos2][0]<=genStartPossitionsList[index1][0]:
            if finalGenPosList==[]:
                finalGenPosList.append((genEndPossitionsList[pos2][0],list(tempDict.keys())))
            else:
                if finalGenPosList[-1][0]<=genEndPossitionsList[pos2][0]:
                    finalGenPosList.append((genEndPossitionsList[pos2][0],list(tempDict.keys())))
            tempDict.pop(genEndPossitionsList[pos2][1],None)
            t2=pos2
        index2=t2    
    
    if finalGenPosList!=[]:
        if finalGenPosList[-1][0]<genStartPossitionsList[pos][0] and len(list(tempDict.keys()))!=len(finalGenPosList[-1][1]):
            finalGenPosList.append((genStartPossitionsList[pos][0],list(tempDict.keys())))            
        elif finalGenPosList[-1][0]<genStartPossitionsList[pos][0]:
            finalGenPosList.append((genStartPossitionsList[pos][0],list(tempDict.keys())))
    tempDict[genStartPossitionsList[pos][1]]=0        
    
    
FinalAdnPosDict=collections.OrderedDict({})
for item in finalGenPosList:
    FinalAdnPosDict[item[0]]=item[1]
#return in FinalAdnPosDict 

#saving the dictionary
import pickle
output=open(genPosDicFile,'wb')
pickle.dump(FinalAdnPosDict,output)
output.close()

Las siguientes funciones realizan el mapeo de posiciones a listas de genes usando el diccionario anterior

In [ ]:
import pickle
#dictionary route
genPosDicFile="D:\\Genomics\\genomePositionsDictionary.pkl" 
#carga del diccionario
pkl_file = open(genPosDicFile, 'rb')
GenomePosDict=pickle.load(pkl_file)
pkl_file.close()

#Basic function for mapping O(n)   
def mapgenome(dictionary,value):
    if dictionary.get(value,None)!=None:
        return dictionary.get(value)
    else:#should change this for something faster. either way ... 0(dict size)
        for key in dictionary.keys():
            if key>value: return dictionary.get(key)
    return 0            

#BinarySearch based function for mapping, O(logn)
def mapgenomeBinarySearch(dictionary, value):
    if dictionary.get(value,None)!=None:
        return dictionary.get(value)
    vmin=0
    vmax=len(dictionary.keys())-1 
    pivot=0
    keyslist=list(dictionary.keys())
    while True:
        pivot1=(vmax+vmin)/2
        pivot2=(vmax+vmin)//2
        if(pivot1==pivot2):
            pivot=pivot2
        else:
            pivot=pivot2+1
        if(vmax-vmin)<=1:
            return dictionary.get(keyslist[pivot])
        if keyslist[pivot]<value:
            vmin=pivot
        else:
            vmax=pivot